In [8]:
import os
import ast
import openai
import time
import json
from json import JSONDecodeError
import re

In [9]:
# openai.organization = ""  #My OpenAI account
# openai.api_key = "" #My Key
# openai.Model.list()

<OpenAIObject list at 0x28de7a7d4f0> JSON: {
  "object": "list",
  "data": [
    {
      "id": "dall-e-3",
      "object": "model",
      "created": 1698785189,
      "owned_by": "system"
    },
    {
      "id": "dall-e-2",
      "object": "model",
      "created": 1698798177,
      "owned_by": "system"
    },
    {
      "id": "gpt-3.5-turbo-0125",
      "object": "model",
      "created": 1706048358,
      "owned_by": "system"
    },
    {
      "id": "text-embedding-ada-002",
      "object": "model",
      "created": 1671217299,
      "owned_by": "openai-internal"
    },
    {
      "id": "tts-1-hd-1106",
      "object": "model",
      "created": 1699053533,
      "owned_by": "system"
    },
    {
      "id": "text-embedding-3-small",
      "object": "model",
      "created": 1705948997,
      "owned_by": "system"
    },
    {
      "id": "gpt-4-0613",
      "object": "model",
      "created": 1686588896,
      "owned_by": "openai"
    },
    {
      "id": "tts-1-hd",
      "object

In [10]:
with open('llm_summary_by_gpt35.json', 'r', encoding='utf-8') as summary:
    summary_data = json.load(summary)
print(len(summary_data))

1200


In [11]:
# summary_data[0]

In [12]:
with open('evidence_corpus.json', 'r', encoding='utf-8') as f:
    evidence_data = json.load(f)
print(len(evidence_data))

8096


In [13]:
# Call GPT3.5 Model 
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo-16k-0613", 
                                 temperature=0, 
                                 max_tokens=1024):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
        format={"type": "json_object"}
    )
    return response.choices[0].message["content"]

In [14]:
delimiter_1 = "#"
delimiter_2 = "##"

system_message = f"""
You are a Fact Checker.\
The user will give you a news paragraph and several evidence abstracts.\
The evidence abstracts may contain information that upholds or contradicts the news article’s central claim. \
Determine whether the news story is factually true or fake based on the evidence abstracts.\ 
The user will put the news paragraph behind {delimiter_1} and the relevant abstracts behind {delimiter_2}. 
"""


In [15]:
generated_result = []
i = 0

In [30]:
for item in summary_data[849:]:
    print(f"\n\nPredicting {i} :-")
    
    news = item['abstractive']
#     print(news)
    
    evidence_list = item['evidence']
    print(f"evidence list: {evidence_list}")
    
    evidence = [item['abstract'] for item in evidence_data if item['cord_uid'] in evidence_list]
    l = len(evidence_list)
    evidence = ''.join(evidence)
#     print(evidence)

    user_message = f"""
    Here is one news paragraph behind {delimiter_1}{news} and {l} relevant abstracts behind {delimiter_2}{evidence}.\
    Determine if these {l} separate statements support or refute the point of view in the news paragraph.\
    Finally provide an answer of 'True' if the news paragraph is factually true or \
    'False' if it is false based on the evidence with reasons. \
    Provide the final output in a JSON format with the keys: prediction and reason.
    """
    
#     user_message = f"""
#     Here is one news paragraph behind {delimiter_1}{news} and {l} relevant abstracts behind {delimiter_2}{evidence}.\
#     Determine in {l} separate statements if these abstracts uphold or contradict the point of view in the news paragraph.\
#     Finally provide a conclusion if the news paragraph is factually True or False based on the evidence.
#     """
  
#     messages =  [  
#                 {'role':'system', 'content': system_message},    
#                 {'role':'user', 'content': user_message},  
#                 ]
    
    assistant_message = """
    As a Fact Checker, carefully analyze the evidence abstracts and provide the result in a JSON format. \
    Reference specific phrases or sentences from both the news story and the evidence to support your prediction.
    """

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message},
        {'role': 'assistant', 'content': assistant_message},
    ]
    
    response = get_completion_from_messages(messages)
    print(response)
    
    result = dict()
    result['news'] = news
    result['evidence list'] = evidence_list
    result['ground truth'] = item['label']
#     result.update(json.loads(response))
    
    try:
        result.update(json.loads(response))
        
    except JSONDecodeError as e:
        pattern = r'"prediction": "(.*?)",\s*"reason": "(.*?)"'
        match = re.search(pattern, response)
        if match:
            result['prediction'] = match.group(1)
            result['reason'] = match.group(2)
#             print(dict({'prediction':result['prediction'], 'reason':result['reason']})) 
    
    generated_result.append(result)
    
    time.sleep(5)
    i += 1
    
#     break



Predicting 849 :-
evidence list: ['k5b6abyk', 'k5b6abyk', 'uxybo6sv']
{
  "prediction": "True",
  "reason": "The evidence abstracts do not directly support or refute the claim made in the news paragraph. The evidence abstracts discuss the expression and distribution pattern of clock genes in nasal mucosa, and the inflammatory response of sinonasal mucosa in rats. These abstracts do not provide any information about the "Catheter-BSI Eliminator" technology or its effectiveness in eliminating catheter-related bloodstream infections in neonates. Therefore, based on the evidence provided, it is not possible to determine the factual accuracy of the news paragraph."
}


Predicting 850 :-
evidence list: ['k6g12iq7', 'k6g12iq7', 'g95p6emb']
{
  "prediction": "False",
  "reason": "The evidence abstracts do not provide any information directly related to the groundbreaking new treatment for sepsis in neonatal care. The abstracts discuss variable-gain control strategies for mechanical ventilato



Predicting 864 :-
evidence list: ['kvvxmvqo', 'hr5g1mvo', 'sy3ri0k9']
{
  "prediction": "False",
  "reason": "The evidence abstracts do not provide any relevant information or support for the claim made in the news paragraph about feline chronic vomiting and gut bacteria imbalance. The evidence abstracts discuss topics such as baculovirus expression vector systems, virus-like particles, and protein complex production in insect cells, which are unrelated to the claim in the news paragraph."
}


Predicting 865 :-
evidence list: ['kx7z79pf', 'v6syqyv4', 'kpy5fesw']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the claim made in the news paragraph that Respiratory Syncytial Virus (RSV) has the ability to change its genetic makeup in response to environmental factors. The first abstract discusses the immunological mechanisms underlying higher susceptibility to infection, which implies the ability of the virus to adapt. The second abstract discusses the induction of

{
  "prediction": "False",
  "reason": "The evidence abstracts do not provide any relevant information or evidence that supports or refutes the claim made in the news paragraph about chest CT scans accurately predicting mortality in COVID-19 cases. The evidence abstracts primarily discuss the use of Newcastle disease virus (NDV) as a vaccine vector and the efficacy of commercially available vaccines against Newcastle disease (ND) and Infectious Bronchitis (IB) in poultry. Therefore, there is no direct connection between the evidence abstracts and the claim made in the news paragraph."
}


Predicting 880 :-
evidence list: ['lmmhf1qn', 'jcpkla55', 'yoff6se5']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the central claim of the news paragraph. The abstracts discuss the different species of bacteria that cause TB and NTM diseases, as well as the differences in disease presentation, diagnostic modalities, and treatment modalities between TB and NTM diseases."
}


P



Predicting 895 :-
evidence list: ['m6wid9v5', 'fdz2dwk6', '1u3s1nx1']
{
  "prediction": "False",
  "reason": "The evidence abstracts do not provide any information that directly supports or refutes the claim made in the news paragraph about a groundbreaking study on a ventilation strategy for obese patients undergoing bariatric surgery. The evidence abstracts primarily discuss hantaviruses and their impact on human health, but they do not provide any information about the ventilation strategy mentioned in the news paragraph. Therefore, based on the evidence provided, the news paragraph is factually false."
}


Predicting 896 :-
evidence list: ['m7d7zrtm', '34hcg3iv', 'x9azxi9m']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the news paragraph's claim that researchers have made a groundbreaking discovery in the treatment of the influenza D virus. The abstracts mention the identification of influenza B as the cause of influenza-like illness (ILI) and rash, and t



Predicting 908 :-
evidence list: ['xqamj7qo', 'mp80hqer', 'n6sq2jz9']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the point of view in the news paragraph. None of the infants tested positive for the virus and none developed serious symptoms. This suggests that infants born to COVID-19 positive mothers may not be at the same risk as adults and older children."
}


Predicting 909 :-
evidence list: ['nz8qvuw6', 'mohqbpqs', 'ncl2p7kg']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the statements made in the news paragraph. The first abstract confirms that the virus was initially found in individuals who had traveled to Wuhan or visited a seafood wholesale market in the city. The second abstract confirms the 96.2% similarity between the virus and bat β-coronaviruses from lineage B. The third abstract confirms that the virus enters target cells by utilizing the angiotensin-converting enzyme 2 receptor, which is also used by the SARS-CoV vir

...

{"prediction": "True", "reason": "The evidence abstracts support the central claim of the news paragraph. The abstracts mention that individuals on antiretroviral therapy (ART) presented with only mild COVID-19 symptoms, suggesting a lower risk of severe disease manifestation. The abstracts also highlight the potential benefits of early initiation of ART in HIV-infected individuals to mitigate the impact of COVID-19 coinfection. These findings align with the information presented in the news paragraph."}


Predicting 1146 :-
evidence list: ['o4k30ki3', 'vqltdvrx', 'ijqwc7dz']
{"prediction": "True", "reason": "The evidence abstracts support the central claim of the news paragraph. The first abstract provides a framework for prioritizing population groups for COVID-19 vaccination, which aligns with the news paragraph's recommendation to prioritize essential workers, older adults, individuals with underlying health conditions, pregnant women, adults without underlying health conditions, a



Predicting 1158 :-
evidence list: ['gu1rmyrs', 'chpzxo43', 'kfsuiz5t']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the claims made in the news paragraph. The abstracts mention that the study conducted in Wuhan City estimated a reproduction number of 5.20, which indicates high transmission potential. It also states that enhanced public health interventions led to a decline in the reproduction number to 0.58. The abstracts also mention that the study estimated 19.1% of the population in Wuhan City was infected, which aligns with the news paragraph's claim. Additionally, the abstracts mention that the study found a higher risk of death than previously reported, further supporting the news paragraph's claim. Therefore, the news paragraph is factually true based on the evidence provided."
}


Predicting 1159 :-
evidence list: ['jm5jblyt', 'pfhfmck1', 'jm5jblyt']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the claims made in the news para



Predicting 1171 :-
evidence list: ['kgl3r43i', '95x4lxla', 'ym3bp5n0']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the central claim of the news paragraph. The abstracts mention that lung damage caused by COVID-19 is reversible in most cases and that there is a concerning trend of recurrent positive results in nucleic acid tests for discharged patients, even without clinical symptoms. This aligns with the news paragraph's claim that the study challenges the current discharge criteria for COVID-19 patients and raises questions about the accuracy of the tests."
}


Predicting 1172 :-
evidence list: ['lykmjvtv', 'pv8djl4a', 'u8ceopa5']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the central claim of the news paragraph. They confirm that adverse outcomes in COVID-19 patients are not solely due to the viral infection itself, but also to bacterial coinfection. The case-control study analyzed 50 cases and 50 controls, revealing that 80% of



Predicting 1185 :-
evidence list: ['01yi70c0', 'yycambht', 'yycambht']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the central claim of the news paragraph. The first evidence abstract states that COVID-19 has long-term effects on various systems, which aligns with the news paragraph's claim of persistent symptoms in COVID-19 survivors. The second evidence abstract mentions a study conducted in Saudi Arabia that found a high proportion of individuals with long-COVID-19 symptoms, further supporting the news paragraph's claim. The third evidence abstract emphasizes the need for proper assessment and rehabilitation for post-COVID syndrome, which is in line with the news paragraph's emphasis on targeted vaccination efforts and rehabilitation for long-term consequences."
}


Predicting 1186 :-
evidence list: ['chzamfb2', '8mokwgbs', '8mokwgbs']
{
  "prediction": "False",
  "reason": "The evidence abstracts contradict the news paragraph. The abstracts state that th



Predicting 1198 :-
evidence list: ['41ogljo8', 'e83kp4aj', '99we37qz']
{
  "prediction": "True",
  "reason": "The evidence abstracts support the news paragraph's claim of alarming levels of COVID-19 exposure in Senegal. The abstracts mention a nationwide cross-sectional epidemiological survey conducted in Senegal, which estimated a national seroprevalence of 28.4%. The abstracts also mention higher rates of exposure in certain regions such as Ziguinchor, Sedhiou, Dakar, and Kaolack. Additionally, the abstracts state that the virus affected all age groups and both symptomatic and asymptomatic individuals. These findings align with the news paragraph's claim and call for urgent preventive and control measures to be implemented in Senegal."
}


Predicting 1199 :-
evidence list: ['qpcvxwti', 's7uop4h8']
{
  "prediction": "False",
  "reason": "The evidence abstracts do not provide any information about a groundbreaking discovery of a miracle drug that can cure COVID-19. The abstracts main

In [49]:
i

0

In [32]:
response

'{\n  "prediction": "False",\n  "reason": "The evidence abstracts do not provide any information about a groundbreaking discovery of a miracle drug that can cure COVID-19. The abstracts mainly discuss the identification and spread of the virus, as well as the epidemiology and pathology of COVID-19. There is no mention of a specific drug or its success rate in curing COVID-19 patients. Therefore, the news paragraph\'s claim is not supported by the evidence."\n}'

In [24]:
# result['prediction'] = 'True'
# result['reason'] = 'The evidence abstracts support the point of view in the news paragraph. Both the news paragraph and the evidence abstracts mention the "two-hit" hypothesis, the role of microglia in the severity of COVID-19, and the hyperactivation of microglia contributing to the severity of the disease. The evidence abstracts also mention the neurological manifestations as a serious cause of severe COVID-19, which aligns with the news paragraph. Therefore, the news paragraph is factually true.'

In [12]:
# evidence_list

In [33]:
result

{'news': "Scientists have reportedly made a groundbreaking discovery by finding a miracle drug that can cure COVID-19. This drug directly targets the virus, inhibiting its replication and enabling the body's immune system to eliminate it. Preliminary results show a near 100% success rate in curing COVID-19 patients, with minimal side effects. However, it is important to approach this news with caution and rely on peer-reviewed studies and expert opinions for validation. Further research and rigorous testing are necessary before accepting any new treatment as a definitive cure for COVID-19.",
 'evidence list': ['qpcvxwti', 's7uop4h8'],
 'ground truth': 0,
 'prediction': 'False',
 'reason': "The evidence abstracts do not provide any information about a groundbreaking discovery of a miracle drug that can cure COVID-19. The abstracts mainly discuss the identification and spread of the virus, as well as the epidemiology and pathology of COVID-19. There is no mention of a specific drug or it

In [34]:
# generated_result.append(result)

In [35]:
len(generated_result)

1200

In [36]:
# [item['abstract'] for item in evidence_data if item['cord_uid'] in evidence_list][0]

In [37]:
# response1 = response

In [38]:
# print(response1)

In [ ]:
generated_result

In [ ]:
os.getcwd()

In [39]:
with open('final_output_llm-gpt35.json', 'w') as file:
    json.dump(generated_result, file)

In [42]:
# with open('final_output_llm-gpt35.json', 'w') as file:
#     json.dump(d3, file)

In [17]:
with open('final_output_llm-gpt35.json', 'r') as file:
    op = json.load(file)

In [50]:
j = 0
for i in op:
    if i['ground truth'] == 0 and i['prediction'] == 'False':
        print(i)
        break
    j+=1

{'news': 'Exercise therapy has been found to be ineffective in improving ambulatory ability in a new model of peripheral arterial disease (PAD), according to a groundbreaking study published in the New England Journal of Medicine. The study used a two-stage procedure in mice, resulting in more severe and sustained ischemia than current models. The mice were unable to walk again after the exercise session, challenging the conventional wisdom on the treatment of PAD. The findings highlight the need for new approaches and therapies for PAD patients.', 'evidence list': ['auvpy0ls', 'l024exll', '68sedl3p'], 'ground truth': 0, 'prediction': 'False', 'reason': 'The evidence abstracts provided do not directly address the central claim of the news paragraph, which is about the ineffectiveness of exercise therapy in improving ambulatory ability in peripheral arterial disease (PAD). The evidence abstracts discuss HIV-1 molecular epidemiology in Nigeria and the genomic variability and epidemiology

In [51]:
j

600

In [52]:
op[600]

{'news': 'Exercise therapy has been found to be ineffective in improving ambulatory ability in a new model of peripheral arterial disease (PAD), according to a groundbreaking study published in the New England Journal of Medicine. The study used a two-stage procedure in mice, resulting in more severe and sustained ischemia than current models. The mice were unable to walk again after the exercise session, challenging the conventional wisdom on the treatment of PAD. The findings highlight the need for new approaches and therapies for PAD patients.',
 'evidence list': ['auvpy0ls', 'l024exll', '68sedl3p'],
 'ground truth': 0,
 'prediction': 'False',
 'reason': 'The evidence abstracts provided do not directly address the central claim of the news paragraph, which is about the ineffectiveness of exercise therapy in improving ambulatory ability in peripheral arterial disease (PAD). The evidence abstracts discuss HIV-1 molecular epidemiology in Nigeria and the genomic variability and epidemio

In [55]:
for item in summary_data[600:]:
#     print(f"\n\nPredicting {i} :-")
    
    news = item['abstractive']
    print(news)
    
    evidence_list = item['evidence']
    print(f"evidence list: {evidence_list}")
    print("\n")
    
    evidence = [item['abstract'] for item in evidence_data if item['cord_uid'] in evidence_list]

    evidence = ''.join(evidence)
    print(evidence)
    break

Exercise therapy has been found to be ineffective in improving ambulatory ability in a new model of peripheral arterial disease (PAD), according to a groundbreaking study published in the New England Journal of Medicine. The study used a two-stage procedure in mice, resulting in more severe and sustained ischemia than current models. The mice were unable to walk again after the exercise session, challenging the conventional wisdom on the treatment of PAD. The findings highlight the need for new approaches and therapies for PAD patients.
evidence list: ['auvpy0ls', 'l024exll', '68sedl3p']


Nigeria has the highest number of AIDS-related deaths in the world. In this study, we characterised the HIV-1 molecular epidemiology by analysing 1442 HIV-1 pol sequences collected 1999–2014 from four geopolitical zones in Nigeria using state-of-the-art maximum-likelihood and Bayesian phylogenetic analyses. The main circulating forms were the circulating recombinant form (CRF) 02_AG (44% of the analy